In [1]:
from peft import LoraConfig,TaskType
import torch
from transformers import AutoModel,AutoTokenizer,AutoModelForCausalLM,DataCollatorForSeq2Seq
from peft import get_peft_model
import pandas as pd 
import datasets
from transformers import Trainer,TrainingArguments
from datasets import load_dataset
from peft import PeftConfig,PeftModel
import os
import deepspeed

/home/ys/anaconda3/envs/chatglm_test/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


[2024-05-18 21:15:23,568] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
#data.output[1]

In [3]:
path  = '/home/ys/zhy/chatglm微调/chat-dataset-baseline-main/history/其他中文问题补充/名词解释类问题.json'

In [4]:
data = pd.read_json(path)

In [5]:
data.drop(columns = ['input'],inplace = True)

In [6]:
import numpy as np
np.where(data.isnull())

(array([], dtype=int64), array([], dtype=int64))

In [7]:
model_name_path = './bloom1b4/'
tokenizer =AutoTokenizer.from_pretrained(model_name_path,trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name_path,
                                                 trust_remote_code=True,low_cpu_mem_usage = True,load_in_8bit = False)
if model_name_path == './qwen7b-chat/':
    tokenizer.pad_token_id = tokenizer.eod_id
    

In [8]:
#query='世界上第二高的山峰是？'
#model.chat(tokenizer,query,history = [])

In [9]:
model

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(46145, 2048)
    (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
  )
  (l

In [10]:
#tokenizer.pad_token

In [11]:
def precess(example):
    context_id = tokenizer.encode(example['instruction'],max_length = max_seq_len,truncation = True)
    target_id = tokenizer.encode(example['output'],max_length = max_seq_len,add_special_tokens = False,truncation = True)
    input_ids = context_id + target_id + [tokenizer.eos_token_id]
    return {'input_ids':input_ids,'context_len':len(context_id),'target_len':len(target_id)}
def data_collator(features:list):
    len_id = [len(x['input_ids']) for x in features]
    longest = max(len_id)
    input_ids = []
    label_ls = []
    for length,feature in sorted(zip(len_id,features),key = lambda x:-x[0]):  # 按照batch中最长的input_ids进行padding
        ids = feature['input_ids']
        context_len = feature['context_len']

        label = (
            [-100]*(context_len - 1) + ids[(context_len - 1):] + [-100]*(longest-length) #  (instruct - > -100)   +target + padding
        )
        ids = ids + [tokenizer.pad_token_id] *(longest-length)
        label_ls.append(torch.LongTensor(label))
        input_ids.append(torch.LongTensor(ids))
    input_ids = torch.stack(input_ids)
    labels = torch.stack(label_ls)
    return {'input_ids':input_ids,'labels':labels}

def process_qwen(example):
    MAX_LENGTH = 512
    input_ids, attention_mask,labels = [],[],[]
    instruction = tokenizer("\n".join(["<|im_start|>system",'你是智能助手，你擅长回答三国时期的问题'+"<|im_end|>" + "\n<|im_start|>user\n" + example["instruction"] + "<|im_end|>\n"])
                            ,truncation = True,
                            add_special_tokens=False)
                        
    #print(example['output'], tokenizer.eos_token,)
    response = tokenizer(
        "<|im_start|>assistant\n" + f"{example['output']}" + "<|im_end|>\n",truncation = True,
    )
    input_ids = instruction['input_ids'] + response['input_ids'] + [tokenizer.pad_token_id]
    attention_mask = instruction['attention_mask'] + response['attention_mask']+ [1]
    labels = [-100] * len(instruction['input_ids']) + response['input_ids'] + [tokenizer.pad_token_id]
    if len(input_ids) > MAX_LENGTH:
        input_idsn = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        'input_ids':input_ids,
        'attention_mask':attention_mask,
        'labels':labels
    }

def process_func(example):
    MAX_LENGTH = 512
    input_ids, attention_mask,labels = [],[],[]
    instruction = tokenizer(example["instruction"] + '\n'
                            ,
                            add_special_tokens=False)
                        
    #print(example['output'], tokenizer.eos_token,)
    response = tokenizer(
     f"{example['output']}" + tokenizer.eos_token,
    )
    input_ids = instruction['input_ids'] + response['input_ids'] 
    attention_mask = instruction['attention_mask'] + response['attention_mask'] 
    labels = [-100] * len(instruction['input_ids']) + response['input_ids']  
    if len(input_ids) > MAX_LENGTH:
        input_idsn = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        'input_ids':input_ids,
        'attention_mask':attention_mask,
        'labels':labels
    }

In [12]:
dftrain = datasets.Dataset.from_pandas(data)
dftrain

Dataset({
    features: ['instruction', 'output'],
    num_rows: 152
})

In [13]:
# df_train = dftrain.map(precess).select_columns(['input_ids', 'context_len', 'target_len'])
df_train = dftrain.shuffle().map(process_func,remove_columns=dftrain.column_names, 
                                 #batched = False
                                ).filter(None)


Map:   0%|          | 0/152 [00:00<?, ? examples/s]

Filter:   0%|          | 0/152 [00:00<?, ? examples/s]

In [14]:
tokenizer.decode(df_train[0]['input_ids'])

'A100是什么？请详细解释：\nA100是英伟达（NVIDIA）推出的一款高性能GPU加速器。它是英伟达推出的第一款基于全新Ampere架构的数据中心GPU加速器，旨在满足人工智能、高性能计算、数据分析等领域的需求。A100采用7nm工艺打造，拥有6912个CUDA核心，核心频率为1.41GHz，具有每秒19.5万亿次的浮点计算能力（FP16）。A100还具备独特的Tensor核心，这种新型计算核心是专门为深度学习框架优化而设计的，可在训练和推理时提供高效的加速，能够提供高达3倍的性能提升。此外，A100还采用了英伟达的第三代NVIDIA NVLink和NVSwitch技术，可提供高达1.6TB/s的内部带宽，实现了GPU之间的直接互连，有效地降低了延迟和增加了吞吐量。A100可以通过NVIDIA的GPU云平台A100 Tensor Core GPU Cloud来使用，也可以集成到数据中心的GPU服务器中，为高性能计算、深度学习等应用提供强大的加速能力。</s>'

In [15]:
model

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(46145, 2048)
    (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
  )
  (l

In [16]:
from peft import LoraConfig,TaskType,get_peft_model
import re 
pattern = r'\((\w+)\): Linear'
linear_layers = re.findall(pattern,str(model.modules))
config = LoraConfig(task_type = TaskType.CAUSAL_LM,lora_alpha=8,r=32,target_modules=list(set(linear_layers)))
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=32, target_modules=['lm_head', 'dense_4h_to_h', 'dense', 'dense_h_to_4h', 'query_key_value'], lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None)

In [17]:
linear_layers

['query_key_value', 'dense', 'dense_h_to_4h', 'dense_4h_to_h', 'lm_head']

In [18]:
model = get_peft_model(model,config)
model.print_trainable_parameters

<bound method PeftModel.print_trainable_parameters of PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BloomForCausalLM(
      (transformer): BloomModel(
        (word_embeddings): Embedding(46145, 2048)
        (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (h): ModuleList(
          (0-23): 24 x BloomBlock(
            (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
            (self_attention): BloomAttention(
              (query_key_value): Linear(
                in_features=2048, out_features=6144, bias=True
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=6144, bias=False)
                )
               

In [19]:
model.enable_input_require_grads()

In [20]:
# deepspeed
import os

os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "9994"  # modify if RuntimeError: Address already in use
os.environ["RANK"] = "0"
os.environ["LOCAL_RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"

# 创建训练文件
args = TrainingArguments(
    output_dir = './',
    per_device_train_batch_size = 1,
    num_train_epochs =10 ,
    weight_decay = 0.1,
    lr_scheduler_type = 'linear',
    learning_rate = 5e-4,
    # fp16 = True,
    remove_unused_columns=True,
    logging_steps=10,
    # deepspeed="./ds_config_zero3.json"
    
)
# 创建训练器
trainer = Trainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = df_train,
    #data_collator = data_collator,,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer,padding=True),
    args = args,
    
)

/home/ys/anaconda3/envs/chatglm_test/lib/python3.10/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(


In [21]:
trainer.train()

You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
[rank0]:[W reducer.cpp:1389] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This flag results in an extra traversal of the autograd graph every iteration,  which can adversely affect performance. If your model indeed never has any unused parameters in the forward pass, consider turning this flag off. Note that this warning may be a false positive if your model has flow control causing later iterations to have unused parameters. (function operator())


Step,Training Loss
10,2.143500
20,1.883500
30,2.116100
40,1.929900
50,2.285500
60,2.036100
70,2.026200
80,2.104500
90,1.916300
100,1.929400


TrainOutput(global_step=1520, training_loss=0.562244786920124, metrics={'train_runtime': 211.8698, 'train_samples_per_second': 7.174, 'train_steps_per_second': 7.174, 'total_flos': 2426726358450176.0, 'train_loss': 0.562244786920124, 'epoch': 10.0})

In [ ]:
dftrain[:2]

In [ ]:
x = tokenizer('政治这个词是什么意思?请详细解释:',return_tensors='pt').to(model.device)

In [ ]:
x

In [ ]:
out = model.generate(**x,max_length = 512,top_k = 0.8,top_p = 0.9,num_beams = 2)   
# 还有min_length,
# 以及采样策略 temperature，topk,topp
# 解码策略 do_sample,num_beam
# 惩罚项 repetition_penalty

In [ ]:
tokenizer.decode(out[0].cpu().numpy().tolist())

In [ ]:
help(model.generate)

In [ ]:
model.chat(tokenizer,'数字信号处理是什么？请详细解释',history = [])

In [ ]:
out 

In [ ]:
tokenizer.decode(out[0])

In [ ]:
type(model)

In [ ]:
model.save_pretrained('lora_chatglm.pt')

In [ ]:
model = AutoModel.from_pretrained(model_name_path,trust_remote_code=True).half().cuda()
# 加载保存的lora  config
PeftConfig.from_pretrained('./lora_chatglm.pt')
# 加载保存的lora层
model = PeftModel.from_pretrained(model,'lora_chatglm.pt',is_trainable = True)

In [ ]:
model_merge = model.merge_and_unload()  # 合并到原模型中

In [ ]:
model_merge.chat(tokenizer,'赵云是如何死的？',history = []) 

In [ ]:
import json


In [ ]:
d = {
    "fp16": {
        "enabled": "auto",
        "loss_scale": 0,
        "loss_scale_window": 1000,
        "initial_scale_power": 16,
        "hysteresis": 2,
        "min_loss_scale": 1
    },

    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": "auto",
            "betas": "auto",
            "eps": "auto",
            "weight_decay": "auto"
        }
    },

    "scheduler": {
        "type": "WarmupLR",
        "params": {
            "warmup_min_lr": "auto",
            "warmup_max_lr": "auto",
            "warmup_num_steps": "auto"
        }
    },

    "zero_optimization": {
        "stage": 2,
        "offload_optimizer": {
            "device": "cpu",
            "pin_memory": True
        },
        "allgather_partitions": True,
        "allgather_bucket_size": 2e8,
        "overlap_comm": True,
        "reduce_scatter": True,
        "reduce_bucket_size": 2e8,
        "contiguous_gradients": True
    },

    "gradient_accumulation_steps": "auto",
    "gradient_clipping": "auto",
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
}

In [ ]:
with open('ds_config_zero3.json', 'w', encoding='utf-8') as f:
    # 使用json.dump()将data字典写入文件
    json.dump(d, f, ensure_ascii=False, indent=4)